<a href="https://colab.research.google.com/github/Herbsta/Blackjack/blob/main/Blackjack_Heuristic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
#Use for card shuffle
import random
import math
 
#Hearts, Diamonds, Clubs, Spade
suits = ('H', 'D', 'C', 'S')
 
#Possible card ranks
ranking = ('A', '2', '3', '4','5', '6', '7', '8', '9', '10', 'J', 'Q','K')
#ranking = ('2','2','2','A','A','A','A','A','A')
 
#Point values dict 
global card_val
card_val = { 'A': 'A', '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, 'J': 10, 'Q': 10, 'K': 10 }
 
class PlayerBank:
  def __init__(self,chip_pool,id):
    self.chip_pool = chip_pool
    self.bet = 0
    self.win = 0
    self.id = id
    self.winpool = 0
 
#Card Class
class Card:
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
        
    def __str__(self):
        return self.suit + self.rank
    
    def grab_suit(self):
        return self.suit
    
    def grab_rank(self):
        return self.rank
    
    def draw(self):
        print(self.suit + self.rank)
        
        
#Hand Class       
class Hand:
    def __init__(self):
        self.cards = []
        self.value = 0
        self.isFinished = False
        self.surrendered = False
        self.isBust = False
        self.blackjack = False
        self.isDoubledDown = False
        
        
    def __str__(self): 
        '''Returns a string of current hand composition'''
        hand_comp = "" 
        
        for card in self.cards:
            card_name = card.__str__()
            hand_comp += " " + card_name 
            
        return 'The hand has %s' %hand_comp
    
    def card_add(self, card,aceAlways11 = False): 
     '''Adds another card to the hand'''
     ace = 0
     self.cards.append(card)
     #calculate the new value
 
     self.value = 0
     for card in self.cards:
       v = card_val[card.rank]
       if 'A' == card.rank:
         ace += 1
         v = 11
            
       self.value += v
 
        
     while True:
       if self.value > 21:
         if ace > 0:
           self.value -= 10
           ace -=1
         else:
           inputField.printStatement("Bust")
           inputField.printStatement(self.__str__())
           self.isBust = True
           self.isFinished = True
           break
       else:
         break
 
        
    def draw(self, hidden):
        if hidden == True:
          #Don't show first hidden card
          starting_card = 1
        else: 
          starting_card = 0
          for x in range(starting_card, len(self.cards)):
            self.cards[x].draw()
                
#Deck Class
class Deck: 
    def __init__(self,numOfDecks):
        '''Creates a deck in order'''
        self.deck = []
        for suit in suits: 
            for rank in ranking: 
                self.deck.append(Card(suit, rank))
        self.deck = self.deck * numOfDecks
        
                
    def shuffle(self): 
        '''Shuffles the deck, without using Python's built-in method'''
        random.shuffle(self.deck)
        
    def deal(self):
        '''Grabs the first item from the deck'''
        single_card = self.deck.pop()
        
        return single_card
    
    def __str__(self):
        deck_comp = "" 
        for card in self.deck: 
            deck_comp += " " + card.__str__()
            
        return "The deck has" + deck_comp
'''
#First Bet
def make_bet(player):
    print("Player ID: {}".format(player.id))
    #Asks the player for the bet amount and 
    player.bet = 0
    player.win = 0
    
    print(' What amount of chips would you like to bet? (Enter whole integer please) ')
    
    #While loop to keep asking for the bet
    while player.bet == 0:
        bet_comp = input() #Use bet_comp as a checker 
        bet_comp = float(bet_comp)
        #Check to make sure the bet is within the remaining amount of chips left
        if bet_comp >= 1 and bet_comp <= player.chip_pool:
            player.bet = bet_comp
        else: 
            print("Invalid bet, you only have " + str(player.chip_pool) + " remaining")
    return player
'''
 
def Split(deck,handArray,id,index):
  hasSplit = False
  rank1 = handArray[id][index].cards[0].rank
  rank2 = handArray[id][index].cards[1].rank
  if rank1 == rank2:
    option = inputField.splitYesNo("Player {}, do you want to split? y/n ".format(id),handArray[id][index])
    if option.lower() == 'y':
      hasSplit = True
      # make 2 new hands and destroy the old one and add those card to the new hands
      cards = handArray[id][index]
      handArray[id].append(Hand())
      handArray[id][-1].card_add(cards.cards[0],aceAlways11 = True)
      handArray[id][-1].card_add(deck.deal(),aceAlways11 = True)
      handArray[id].append(Hand())
      handArray[id][-1].card_add(cards.cards[1],aceAlways11 = True)
      handArray[id][-1].card_add(deck.deal(),aceAlways11 = True)
      handArray[id].pop(index)

      if rank1 == 'A':
        handArray[id][-1].isFinished = True
        handArray[id][-2].isFinished = True
    
  return deck, handArray, hasSplit
 
            
def deal_cards(players,deckCount,maxSplitDecks,doubleTotals):
    '''This function deals out cards and sets up round''' 
    #set up dictionaries for player's hand(s)
    handDict = {}
    for i in players:
      handDict[i.id] = []
    
 
    #Create a deck
    deck = Deck(deckCount)
    #Shuffle it
    deck.shuffle()
    #Set up bet
    for x,i in enumerate(players):
      players[x] = inputField.make_bet(i)
            
    #Set up both player and dealer hands 
 
    for i in players:
      handDict[i.id].append(Hand())
 
    dealer_hand = Hand()
    
    #Deal out initial cards 
    for i in players:
      handDict[i.id][0].card_add(deck.deal(),aceAlways11 = True)
      handDict[i.id][0].card_add(deck.deal(),aceAlways11 = True)
      
      #print draw
      inputField.printStatement("Player {}'s Hand: ".format(i.id))
      inputField.printStatement(handDict[i.id][0].__str__())
 
    
    dealer_hand.card_add(deck.deal(), aceAlways11 = True)
    dealer_hand.card_add(deck.deal(), aceAlways11 = True)
 
    inputField.dealer_hand_1st_card = dealer_hand.cards[0]
    
    inputField.printStatement("Dealer's Hand")
    inputField.printStatement(dealer_hand.cards[0].__str__())
 
    #check for natural
    for i in players:
      if handDict[i.id][0].value == 21:
        handDict[i.id][0].isFinished = True
        handDict[i.id][0].blackjack = True
    
    if dealer_hand.value == 21:
      dealer_hand.blackjack = True
      dealer_hand.isFinished = True
 
 
    #check for early surrender
    for i in players:
      if handDict[i.id][0].isFinished:
        continue
      option = inputField.surrenderYesNo("Player {}, do you want to surrender? y/n".format(i.id),handDict[i.id][0])
      if option.lower() == 'y':
        handDict[i.id][0].isFinished = True
        handDict[i.id][0].surrendered = True
 
 
    for i in players:
      # Check for split
      #skips natural blacks
      if handDict[i.id][0].isFinished:
        continue
 
      splitCount = 1
      j = 0
      while j < len(handDict[i.id]):
        #can't split if they dont have enough money
        if i.bet * 2 > i.chip_pool or handDict[i.id][j].isFinished:
          break
        deck,handDict, hasSplit = Split(deck,handDict,i.id,j)
        if hasSplit:
          inputField.printStatement("Split Hand is: ",end="")
          inputField.printStatement(handDict[i.id][j].__str__())
          splitCount += 1
          if splitCount == maxSplitDecks: break
        else:
          j += 1
 
 
    # Check for double
    for i in players:
      for x,j in enumerate(handDict[i.id]):
        #skips natural blacks
        if j.isFinished:
          continue
        canDouble = False
        if i.bet * 2 > i.chip_pool:
          continue
 
        if doubleTotals == "All":
          canDouble = True
        else:
 
          for k in doubleTotals:
            if j.value == k:
              canDouble = True
              break
        
        if canDouble:
          option = inputField.doubleYesNo("Player {}, do you want to double down? y/n".format(i.id),handDict[i.id][x])
          if option.lower() == "y":
            handDict[i.id][x].card_add(deck.deal())
            handDict[i.id][x].isFinished = True
            handDict[i.id][x].isDoubledDown = True
 
    #Hit or Stay Check
 
    for i in players:
      for x,j in enumerate(handDict[i.id]):
        while handDict[i.id][x].isFinished == False:
          
 
          inputField.printStatement("Player {}'s hand: ".format(i.id),end="")
          inputField.printStatement(j.__str__())
          option = inputField.stayHitInput("Player {}, do you want to stay or hit? s/h".format(i.id),j)
          if option.lower() == 's':
            handDict[i.id][x].isFinished = True
          else:
            handDict[i.id][x].card_add(deck.deal(),aceAlways11=False)
 
          if handDict[i.id][x].value == 21:
            handDict[i.id][x].isFinished = True
 
    inputField.printStatement(" ")
    #Dealer's Go
 
    while dealer_hand.isFinished == False:
      #check for soft 17
      if dealer_hand.value < 17:
        dealer_hand.card_add(deck.deal(),aceAlways11=True)
      else:
        dealer_hand.isFinished = True
    
    inputField.printStatement("Dealer's Hand: ")
    inputField.printStatement(dealer_hand.__str__())
 
    #Compare hands and winnings
    for x,i in enumerate(players):
      for y,j in enumerate(handDict[i.id]):
 
        #did double down?
        if j.isDoubledDown:
          dd = 2
        else:
          dd = 1
        
        #check for surrender
        if j.surrendered:
          players[x].win -= players[x].bet /2
          continue
          
        #check if busted
        if j.isBust:
          '''no winnings'''
          players[x].win -= players[x].bet * dd
          
        elif dealer_hand.blackjack:
          #check for dealer blackjack
          if j.blackjack or j.value == 21:
            '''push'''
            
          else:
            '''dealer won'''
            players[x].win -= players[x].bet * dd
            
        elif j.blackjack:
          '''player blackjack'''
          players[x].win += players[x].bet * 1.5
        
        elif dealer_hand.isBust:
          ''' Dealer busted, so player wins'''
          players[x].win += players[x].bet * dd
          
        elif dealer_hand.value < j.value:
          '''player has higher value than dealer'''
          players[x].win += players[x].bet * dd
        elif j.value < dealer_hand.value:
          '''dealer has higher value than player'''
          players[x].win -= players[x].bet * dd
 
        elif j.value == dealer_hand.value:
          '''push'''
      
      players[x].chip_pool += players[x].win   
 
    if inputField.isAuto == False:
 
      for i in players:
        for j in handDict[i.id]:
          print("Player {}".format(i.id))
          print(j.__str__() + " Value: " + str(j.value))
          print("----------")
        
      print("Dealer's Hand: ")
      print(dealer_hand.__str__() + " Value: " + str(dealer_hand.value))
      print("----------")
    return players
 
def PrintWinnings(players):
  for i in players:
    print("Player {} collection: {}".format(i.id,i.win))
    print("Player {} new chip pool: {}".format(i.id,i.chip_pool))

In [36]:
class Input:

  def __init__(self):
    self.isAuto = False
    self.dealer_hand_1st_card = None
    return
  
  def printStatement(self,string,end = "\n"):
    print(string,end=end)
  

In [37]:
class HeuristicInput(Input):
  
  def __init__(self):
    self.isAuto = False
    return
  
  def yesNoInput(self,string,hand):
    option = input(string)
    return option
  
  def surrenderYesNo(self,string,hand):
    return self.yesNoInput(string)

  def splitYesNo(self,string,hand):
    return self.yesNoInput(string)

  def doubleYesNo(self,string,hand):
    return self.yesNoInput(string)
   
  def stayHitInput(self,string,hand):
    option = input(string)
    return option
  

def make_bet(self,player): 
  print("Player ID: {}".format(player.id)) 
  '''Asks the player for the bet amount and ''' 
  player.bet = 0 
  player.win = 0
  print(' What amount of chips would you like to bet? (Enter whole integer please) ')
 
  #While loop to keep asking for the bet
  while player.bet == 0:
    bet_comp = input() #Use bet_comp as a checker 
    bet_comp = float(bet_comp)
    #Check to make sure the bet is within the remaining amount of chips left
    if bet_comp >= 1 and bet_comp <= player.chip_pool:
      player.bet = bet_comp
    else: 
      print("Invalid bet, you only have " + str(player.chip_pool) + " remaining")
  return player


In [38]:
class AutoInput(Input):
  def __init__(self,df):
    super().__init__()
    self.isAuto = True
    self.df = df
    self.hit = "H"
    self.stand = "S"
    self.split = "P"
    self.doubleHit = "D"
    self.doubleStand = "Ds"
    self.splitDoubleHit = "Hs"
    self.surrenderHit = "Sh"
    self.surrenderStand = "Ss"
    return
  
  
  def indexDF(self,hand):
    
    if len(hand.cards) == 2:
      try:
        card1 = str(card_val[hand.cards[0].rank])
        card2 = str(card_val[hand.cards[1].rank])

        if card2 == 'A':
          c = card1
          card1 = card2
          card2 = c

        index = card1 + ',' + card2
        
        
        r = self.df.loc[index,card_val[self.dealer_hand_1st_card.rank]]
        
        return r 
      except KeyError:
        pass
    
    try:
      index = hand.value
      
      r = self.df.loc[index,card_val[self.dealer_hand_1st_card.rank]]
      
    except KeyError:
      if hand.value > 17:
        
        return 'S'
      elif hand.value < 8:
        return 'H'
      else:
        print("WTF")
        
 
  
  def printStatement(self,string,end='\n'):
    return
  
  def surrenderYesNo(self,string,hand):
    v = self.indexDF(hand)
    if v == self.surrenderHit or v == self.surrenderStand:
      return 'y'
    else:
      return 'n'
  
  def splitYesNo(self,string,hand):
    v = self.indexDF(hand)
    if v == self.split or v == self.splitDoubleHit:
      return 'y'
    else:
      return 'n'
  
  def doubleYesNo(self,string,hand):
    v = self.indexDF(hand)
    if v == self.doubleHit or v == self.doubleStand:
      return 'y'
    else:
      return 'n'
  
  def stayHitInput(self,string,hand):
    v = self.indexDF(hand)
    if v == self.stand or v == self.doubleStand or v == self.surrenderStand:
      return 's'
    else:
      return 'h'
  
  def make_bet(self,player):
    '''
    player.winpool += player.win
    if player.winpool > 0:
      player.bet = math.ceil(player.winpool * 0.7)
    else:
      player.bet = 2
    '''
    player.bet = 1
    #player.bet = math.ceil(player.chip_pool * 0.02)
      
      
      
       
    
    player.win = 0
    return player


In [39]:
!pip install --upgrade -q pygsheets

In [40]:
import google.auth
from google.colab import auth

auth.authenticate_user()
import pygsheets
credentials, _ = google.auth.default()
gc = pygsheets.client.Client(credentials)

In [41]:
key = "BlackJack_Basic_Strat"
sheet = gc.open(key)

In [42]:
worksheet = sheet.worksheet_by_title("Sheet1")

In [43]:
import pandas as pd

In [44]:
df = worksheet.get_as_df()
df = df.set_index("Index")

In [45]:
display(df)

,2,3,4,5,6,7,8,9,10,A
Index,,,,,,,,,,
8,H,H,H,H,H,H,H,H,H,H
9,H,D,D,D,D,H,H,H,H,H
10,D,D,D,D,D,D,D,D,H,H
11,D,D,D,D,D,D,D,D,D,H
12,H,H,S,S,S,H,H,H,H,H
13,S,S,S,S,S,H,H,H,H,H
14,S,S,S,S,S,H,H,H,H,H
15,S,S,S,S,S,H,H,H,Sh,Sh
16,S,S,S,S,S,H,H,Sh,Sh,Sh


In [46]:
try: 
  d = df.loc['A,A']
except KeyError:
  print("Not in Key")

In [47]:
df.loc[16,6]

'S'

In [48]:
'''The following code will initiate the game.''' 
#doubleTotals = "All" #for double down on any 2 1st cards

doubleTotals = [9,10,11] 
DeckNumber = 8
reSplitNumber = 4
startChips = 10000000
rounds = 50000
playerNumber = 1

global inputField
inputField = AutoInput(df)
inputField.isAuto = True
#inputField = HeuristicInput()
w = [0,0,0]
players = []
for i in range(playerNumber): 
  players.append(PlayerBank(startChips,i)) 
for i in range(rounds): 
  players = deal_cards(players,DeckNumber,reSplitNumber,doubleTotals) 

  if inputField.isAuto == False:
    PrintWinnings(players)

  for x,i in enumerate(players):
    if i.chip_pool < 1:
      players.remove(i)
      print("Player {} is broke".format(x))
    if i.win > 0:
      w[0] +=1
    elif i.win == 0:
      w[1] += 1
    else:
      w[2] += 1
  
  if len(players) == 0:
    break



In [49]:
PrintWinnings(players)

Player 0 collection: -1
Player 0 new chip pool: 9995777.5


In [50]:
total = 0
for i in players:
  total += i.chip_pool

print("{}%".format((total / (playerNumber * startChips) * 100)))

99.957775%


In [51]:
print ("{} : {} : {}".format(w[0]/rounds,w[1]/rounds,w[2]/rounds))

0.40204 : 0.09092 : 0.50704
